In [1]:
import numpy as np
import pandas as pd
from scipy import misc

import os
import shutil
from tqdm import tqdm 

from PIL import Image

In [2]:
# the folder for images
dir_train = 'E:/Jupyter NB/10 class Classification/Images'

# the directory of separeated images
image_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test'

# This cell is for testing

In [3]:
'''
for root, dirs, files in os.walk(dir_train):
    print('---root---')
    print(root)
    print('---dirs---')
    print(dirs)
    print('---files---')
    print(files)
    print('---------------------')



subdirs = list(os.walk(dir_train))[1:]

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('\\')[-1]
    print(dir_name)
'''

"\nfor root, dirs, files in os.walk(dir_train):\n    print('---root---')\n    print(root)\n    print('---dirs---')\n    print(dirs)\n    print('---files---')\n    print(files)\n    print('---------------------')\n\n\n\nsubdirs = list(os.walk(dir_train))[1:]\n\nfor dir_path, _, files in tqdm(subdirs):\n    \n    dir_name = dir_path.split('\\')[-1]\n    print(dir_name)\n"

In [4]:
subdirs = list(os.walk(dir_train))[1:]

# collect train metadata
train_metadata = []

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('\\')[-1]
    
    for file_name in files:
        if not file_name.startswith('.'):
            # read image
            temp = misc.imread(os.path.join(dir_path, file_name)) 
            # collect image metadata
            image_metadata = []
            image_metadata.extend([dir_name, file_name])
            image_metadata.extend( 
                list(temp.shape) if len(temp.shape) == 3 
                else [temp.shape[0], temp.shape[1], 1]
            )
            image_metadata.extend([temp.nbytes, temp.dtype])
            # append image metadata to list
            train_metadata.append(image_metadata)

#print(train_metadata)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\aRnob\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.45it/s]


In [5]:
M = pd.DataFrame(train_metadata)
M.columns = ['directory', 'img_name', 'height', 'width', 'channels', 'byte_size', 'bit_depth']

M['category_name'] = M.directory.apply(lambda x: x.split('.')[-1].lower())
M['img_extension'] = M.img_name.apply(lambda x: x.split('.')[-1])
M['category_number'] = M.directory.apply(lambda x: int(x.split('.')[0]))

In [6]:
# number of grayscale images
(M.channels != 3).sum()

8

In [7]:
# number of categories
M.category_name.nunique()

10

In [8]:
# class number -> class name
decode = {n: i for i, n in M.groupby('category_name').category_number.first().iteritems()}
decode

np.save('decode.npy', decode)

In [9]:
# 20 images per class
V = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
V.sort_index(inplace=True)
M.drop(V.index, axis=0, inplace=True)

# 20 images per class for test
Test = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
Test.sort_index(inplace=True)
M.drop(Test.index, axis=0, inplace=True)

In [10]:
# train data
len(M)

1233

In [11]:
# validation data
len(V)

200

In [12]:
#Test data
len(Test)

200

In [13]:
M.to_csv('train_metadata.csv', index=False)
V.to_csv('val_metadata.csv', index=False)
Test.to_csv('test_data.csv', index=False)

In [14]:
t = pd.read_csv('E:/Jupyter NB/10 class Classification/train_metadata.csv')
v = pd.read_csv('E:/Jupyter NB/10 class Classification/val_metadata.csv')
test = pd.read_csv('E:/Jupyter NB/10 class Classification/test_data.csv')

In [15]:
os.mkdir(image_dir + '/train')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/train/' + str(i))

In [16]:
os.mkdir(image_dir + '/val')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/val/' + str(i))

In [17]:
os.mkdir(image_dir + '/test')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/test/' + str(i))

# Train images

In [18]:
train_size = len(t)
train_size

1233

In [19]:
# RGB images
for i, row in tqdm(t.loc[t.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'train', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

1228it [00:07, 161.22it/s]


In [20]:
# grayscale images
for i, row in tqdm(t.loc[t.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'train', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

5it [00:01,  2.70it/s]


# Validation images

In [21]:
val_size = len(v)
val_size

200

In [22]:
# RGB images
for i, row in tqdm(v.loc[v.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'val', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

198it [00:01, 158.01it/s]


In [23]:
# grayscale images
for i, row in tqdm(v.loc[v.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'val', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

2it [00:00, 71.61it/s]


# Test images

In [24]:
test_size = len(test)
test_size

200

In [25]:
# RGB images
for i, row in tqdm(test.loc[test.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'test', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

199it [00:01, 145.60it/s]


In [26]:
# grayscale images
for i, row in tqdm(test.loc[test.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'test', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

1it [00:00, 24.44it/s]
